### Project Gutenberg:  Topic Modelling Using Unsupervised NLP

https://www.gutenberg.org/

Assignment 2: Devika Pace

[Import Libaries](#0)

  [Select and load book(s) from Project Gutenberg](#1)
  - Alice In Wonderland
  - Grimm's Fairy Tales
  - Walden 

[Pre-process Text](#2)
- Clean
- Tokenize
- Stop Word Removal
- Stemming and Lemmatization

[Vectorization](#3)
- Count Vectorizer
- Tf-idf Vectorizer
- Sparsity

[Topic Model with Unsupervised Clustering Algorithm](#4)
- LDA
- NNMF
- LSI

[Metrics & Results](#5)
- Tuning
- Visualization
- Performance


<a name="0"></a>
### **Import Libraries**

In [3]:
import os
from urllib.request import urlopen
from warnings import simplefilter
simplefilter('ignore')

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer  
               
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

<a name="1"></a>
### **Project Gutenberg**

We will download a few different texts for comparison's sake.

In [4]:
url_alice = "https://gutenberg.org/files/11/11-0.txt"      # Alice in Wonderland
url_grimm = "https://gutenberg.org/files/2591/2591-0.txt"  # Grimm's Fairy Tales
url_walden = "https://gutenberg.org/files/205/205-0.txt"   # Walden

def get_gutenberg_text(url):

  with urlopen(url) as response:
    html_content = response.read()
  encoding = response.headers.get_content_charset('utf-8')
  text = html_content.decode(encoding)
  return text

**Alice in Wonderland**

In [5]:
text = get_gutenberg_text(url_alice)
print(f'length of text response: {len(text)}, length of split: {len(text.split("***"))}\n')
text.split('***')

length of text response: 167808, length of split: 7



['\ufeffThe Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: Alice’s Adventures in Wonderland\r\n\r\nAuthor: Lewis Carroll\r\n\r\nRelease Date: January, 1991 [eBook #11]\r\n[Most recently updated: October 12, 2020]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\nProduced by: Arthur DiBianca and David Widger\r\n\r\n',
 ' START OF THE PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ',
 '\r\n\r\n[Illustration]\r\n\r\n\r\n\r\n\r\nAlice’s Adventures in Wonderl

In [6]:
alice = text.split('***')[2]
print(f'LENGTH OF ALIC TEXT: {len(alice)}')
print(f'\n************************START OF ALICE TEXT*****************************:{alice[0:3000]}')
print(f'\n**************************END OF ALICE TEXT*****************************:\n\n{alice[-1000:]}')

LENGTH OF ALIC TEXT: 147994

************************START OF ALICE TEXT*****************************:

[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I.     Down the Rabbit-Hole
 CHAPTER II.    The Pool of Tears
 CHAPTER III.   A Caucus-Race and a Long Tale
 CHAPTER IV.    The Rabbit Sends in a Little Bill
 CHAPTER V.     Advice from a Caterpillar
 CHAPTER VI.    Pig and Pepper
 CHAPTER VII.   A Mad Tea-Party
 CHAPTER VIII.  The Queen’s Croquet-Ground
 CHAPTER IX.    The Mock Turtle’s Story
 CHAPTER X.     The Lobster Quadrille
 CHAPTER XI.    Who Stole the Tarts?
 CHAPTER XII.   Alice’s Evidence




CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pic

**Grimm's Fairy Tales**

In [7]:
text = get_gutenberg_text(url_grimm)
print(f'length of text response: {len(text)}, length of split: {len(text.split("***"))}\n')
text.split('***')

length of text response: 549750, length of split: 6



['\ufeffThe Project Gutenberg eBook of Grimms’ Fairy Tales, by Jacob Grimm and Wilhelm Grimm\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: Grimms’ Fairy Tales\r\n\r\nAuthor: Jacob Grimm and Wilhelm Grimm\r\n\r\nTranslators: Edgar Taylor and Marian Edwardes\r\n\r\nRelease Date: April, 2001 [eBook #2591]\r\n[Most recently updated: June 28, 2021]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\nProduced by: Emma Dudding, John Bickers, Dagny and David Widger\r\n\r\n',
 ' START OF THE PROJECT GUTENBERG EBOOK GRIMMS’ FAIRY TALES ',
 '\r\n\r\n\r

In [8]:
grimm = text.split('***')[2]
grimm_about = text.split('***')[3]
print(f'LENGTH OF GRIMM TEXT: {len(grimm)}')
print(f'\n************************START OF GRIMM TEXT*****************************:{grimm[0:3000]}')
print(f'\n*******************END OF GRIMM TEXT (SNOW WHITE)***********************:\n\n{grimm[-1000:]}')
print(f'******************************ABOUT*************************************:\n{grimm_about}')

LENGTH OF GRIMM TEXT: 528873

************************START OF GRIMM TEXT*****************************:




Grimms’ Fairy Tales

By Jacob Grimm and Wilhelm Grimm



PREPARER’S NOTE

     The text is based on translations from
     the Grimms’ Kinder und Hausmärchen by
     Edgar Taylor and Marian Edwardes.




CONTENTS:

     THE GOLDEN BIRD
     HANS IN LUCK
     JORINDA AND JORINDEL
     THE TRAVELLING MUSICIANS
     OLD SULTAN
     THE STRAW, THE COAL, AND THE BEAN
     BRIAR ROSE
     THE DOG AND THE SPARROW
     THE TWELVE DANCING PRINCESSES
     THE FISHERMAN AND HIS WIFE
     THE WILLOW-WREN AND THE BEAR
     THE FROG-PRINCE
     CAT AND MOUSE IN PARTNERSHIP
     THE GOOSE-GIRL
     THE ADVENTURES OF CHANTICLEER AND PARTLET
       1. HOW THEY WENT TO THE MOUNTAINS TO EAT NUTS
       2. HOW CHANTICLEER AND PARTLET WENT TO VISIT MR KORBES
     RAPUNZEL
     FUNDEVOGEL
     THE VALIANT LITTLE TAILOR
     HANSEL AND GRETEL
     THE MOUSE, THE BIRD, AND THE SAUSAGE
     MOTHER HOLLE


**Walden**


In [9]:
text = get_gutenberg_text(url_walden)
print(f'length of text response: {len(text)}, length of split: {len(text.split("***"))}\n')
text.split('***')

length of text response: 664577, length of split: 7



['\ufeffThe Project Gutenberg eBook of Walden, by Henry David Thoreau\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: Walden\r\n\r\nAuthor: Henry David Thoreau\r\n\r\nRelease Date: January, 1995 [eBook #205]\r\n[Most recently updated: January 28, 2021]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\nProduced by: Judith Boss, and David Widger\r\n\r\n',
 ' START OF THE PROJECT GUTENBERG EBOOK WALDEN ',
 '\r\n\r\n\r\n\r\n\r\nWALDEN\r\n\r\n\r\n\r\n\r\nand\r\n\r\n\r\n\r\nON THE DUTY OF CIVIL DISOBEDIENCE\r\n\r\n\r\n\r\nby Henry David Thoreau\r\

In [10]:
walden = text.split('***')[2]
print(f'LENGTH OF ALIC TEXT: {len(alice)}')
print(f'\n************************START OF THOREAU TEXT*****************************:{walden[0:2500]}')
print(f'\n**************************END OF THOREAU TEXT*****************************:\n\n{walden[-1000:]}')

LENGTH OF ALIC TEXT: 147994

************************START OF THOREAU TEXT*****************************:




WALDEN




and



ON THE DUTY OF CIVIL DISOBEDIENCE



by Henry David Thoreau


cover


Contents


 WALDEN

 Economy
 Where I Lived, and What I Lived For
 Reading
 Sounds
 Solitude
 Visitors
 The Bean-Field
 The Village
 The Ponds
 Baker Farm
 Higher Laws
 Brute Neighbors
 House-Warming
 Former Inhabitants and Winter Visitors
 Winter Animals
 The Pond in Winter
 Spring
 Conclusion

 ON THE DUTY OF CIVIL DISOBEDIENCE



WALDEN

Economy

When I wrote the following pages, or rather the bulk of them, I lived
alone, in the woods, a mile from any neighbor, in a house which I had
built myself, on the shore of Walden Pond, in Concord, Massachusetts,
and earned my living by the labor of my hands only. I lived there two
years and two months. At present I am a sojourner in civilized life
again.

I should not obtrude my affairs so much on the notice of my readers if
very particular inquirie

<a name="2"></a>
### **Pre-process Text**

In [11]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

def preprocess_text(text):

  n = 25
  # clean text (to lowercase, remove punctuation)
  text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

  # tokenize text
  words = word_tokenize(text)
  print('tokenized:\n', words[:n])

  # remove stopwords
  words = [w for w in words if w not in stopwords.words('english')]
  #print('stop words:\n', stopwords.words('english'), '\n')
  print('no stop words:\n', words[:n])

  # Lemmatize verbs by specifying pos
  lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
  print('lemmed with pos:\n', lemmed[:n])

  # stemming - remove prefix, suff
  stemmed = [PorterStemmer().stem(w) for w in lemmed]
  print('stemmed:\n', stemmed[:n])

  return stemmed

print('stop words:\n', stopwords.words('english'), '\n')

stop words:
 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 's

In [12]:
post_alice = preprocess_text(alice)

tokenized:
 ['illustration', 'alice', 's', 'adventures', 'in', 'wonderland', 'by', 'lewis', 'carroll', 'the', 'millennium', 'fulcrum', 'edition', '3', '0', 'contents', 'chapter', 'i', 'down', 'the', 'rabbit', 'hole', 'chapter', 'ii', 'the']
no stop words:
 ['illustration', 'alice', 'adventures', 'wonderland', 'lewis', 'carroll', 'millennium', 'fulcrum', 'edition', '3', '0', 'contents', 'chapter', 'rabbit', 'hole', 'chapter', 'ii', 'pool', 'tears', 'chapter', 'iii', 'caucus', 'race', 'long', 'tale']
lemmed with pos:
 ['illustration', 'alice', 'adventure', 'wonderland', 'lewis', 'carroll', 'millennium', 'fulcrum', 'edition', '3', '0', 'content', 'chapter', 'rabbit', 'hole', 'chapter', 'ii', 'pool', 'tear', 'chapter', 'iii', 'caucus', 'race', 'long', 'tale']
stemmed:
 ['illustr', 'alic', 'adventur', 'wonderland', 'lewi', 'carrol', 'millennium', 'fulcrum', 'edit', '3', '0', 'content', 'chapter', 'rabbit', 'hole', 'chapter', 'ii', 'pool', 'tear', 'chapter', 'iii', 'caucu', 'race', 'long', '

In [13]:
post_grimm = preprocess_text(grimm)

tokenized:
 ['grimms', 'fairy', 'tales', 'by', 'jacob', 'grimm', 'and', 'wilhelm', 'grimm', 'preparer', 's', 'note', 'the', 'text', 'is', 'based', 'on', 'translations', 'from', 'the', 'grimms', 'kinder', 'und', 'hausm', 'rchen']
no stop words:
 ['grimms', 'fairy', 'tales', 'jacob', 'grimm', 'wilhelm', 'grimm', 'preparer', 'note', 'text', 'based', 'translations', 'grimms', 'kinder', 'und', 'hausm', 'rchen', 'edgar', 'taylor', 'marian', 'edwardes', 'contents', 'golden', 'bird', 'hans']
lemmed with pos:
 ['grimms', 'fairy', 'tales', 'jacob', 'grimm', 'wilhelm', 'grimm', 'preparer', 'note', 'text', 'base', 'translations', 'grimms', 'kinder', 'und', 'hausm', 'rchen', 'edgar', 'taylor', 'marian', 'edwardes', 'content', 'golden', 'bird', 'hans']
stemmed:
 ['grimm', 'fairi', 'tale', 'jacob', 'grimm', 'wilhelm', 'grimm', 'prepar', 'note', 'text', 'base', 'translat', 'grimm', 'kinder', 'und', 'hausm', 'rchen', 'edgar', 'taylor', 'marian', 'edward', 'content', 'golden', 'bird', 'han']


In [14]:
post_walden = preprocess_text(walden)

tokenized:
 ['walden', 'and', 'on', 'the', 'duty', 'of', 'civil', 'disobedience', 'by', 'henry', 'david', 'thoreau', 'cover', 'contents', 'walden', 'economy', 'where', 'i', 'lived', 'and', 'what', 'i', 'lived', 'for', 'reading']
no stop words:
 ['walden', 'duty', 'civil', 'disobedience', 'henry', 'david', 'thoreau', 'cover', 'contents', 'walden', 'economy', 'lived', 'lived', 'reading', 'sounds', 'solitude', 'visitors', 'bean', 'field', 'village', 'ponds', 'baker', 'farm', 'higher', 'laws']
lemmed with pos:
 ['walden', 'duty', 'civil', 'disobedience', 'henry', 'david', 'thoreau', 'cover', 'content', 'walden', 'economy', 'live', 'live', 'read', 'sound', 'solitude', 'visitors', 'bean', 'field', 'village', 'ponds', 'baker', 'farm', 'higher', 'laws']
stemmed:
 ['walden', 'duti', 'civil', 'disobedi', 'henri', 'david', 'thoreau', 'cover', 'content', 'walden', 'economi', 'live', 'live', 'read', 'sound', 'solitud', 'visitor', 'bean', 'field', 'villag', 'pond', 'baker', 'farm', 'higher', 'law']


<a name="3"></a>
### **Vectorize Text**

In [15]:
len(post_alice), len(post_grimm), len(post_walden)

(12306, 44860, 54684)

In [16]:
vectorizer = CountVectorizer(analyzer='word', min_df=10,
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

tfidf_vectorizer = TfidfVectorizer(analyzer='word', min_df=10,  
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

vec_alice = vectorizer.fit_transform(post_alice)
vec_tfidf_alice = tfidf_vectorizer.fit_transform(post_alice)

vec_grimm = vectorizer.fit_transform(post_grimm)
vec_tfidf_grimm = tfidf_vectorizer.fit_transform(post_grimm)

vec_walden = vectorizer.fit_transform(post_walden)
vec_tfidf_walden = tfidf_vectorizer.fit_transform(post_walden)

In [17]:
vecs = {'vec_alice': vec_alice, 'vec_tfidf_alice':vec_tfidf_alice, 'vec_grimm':vec_grimm, 
        'vec_tfidf_grimm': vec_tfidf_grimm, 'vec_walden': vec_walden, 'vec_tfidf_walden':vec_tfidf_walden}

#### Check Sparsity

In [18]:
def check_sparsity(key,vec):
    #materialize the sparse data
    data_dense = vec.todense()
    # compute sparsity as % of non-zero cells
    print(key, "sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [19]:
for k,v in vecs.items():
    check_sparsity(k,v)

vec_alice sparsicity:  0.22439548093464512 %
vec_tfidf_alice sparsicity:  0.22439548093464512 %
vec_grimm sparsicity:  0.0934931314461793 %
vec_tfidf_grimm sparsicity:  0.0934931314461793 %
vec_walden sparsicity:  0.05316807626200962 %
vec_tfidf_walden sparsicity:  0.05316807626200962 %


<!-- Choose least sparse ('Alice in Wonderland') vs most sparse ('Walden') to compare results.  We will start with Alice as it is ~20-25% size. -->

We will choose 'Alice in Wonderland' as it is the least sparse and also ~20-25% the size of most sparse ('Walden').

In [20]:
data_vectorized = vec_tfidf_alice.copy()

<a name="4"></a>
### Build Models

In [21]:
# Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=10)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=10)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

# Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=10, max_iter=10, learning_method='online') # no of components = no. of topics
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

(12306, 10)
(12306, 10)
(12306, 10)


In [22]:
# inspect the inferred topics
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [23]:
# check log-likelihood and perplexity of LDA model
def print_performance(model, data_vectorized):   
    # log likelihood: Higher the better
    print("Log Likelihood: ", model.score(data_vectorized))
    # perplexity: lower the better, perplexity = exp(-1. * log-likelihood per word)
    print("Perplexity: ", model.perplexity(data_vectorized))
    # model parameters
    pprint(model.get_params())

In [24]:
print("NMF Model:")
print_topics(nmf_model, tfidf_vectorizer)
print("=" * 20)

NMF Model:
Topic 0:
[('commenc', 4.802616494093945), ('deliber', 0.0), ('bear', 0.0), ('berri', 0.0), ('beneath', 0.0), ('bend', 0.0), ('belong', 0.0), ('bell', 0.0), ('believ', 0.0), ('behold', 0.0)]
Topic 1:
[('accord', 4.469338245807232), ('deliber', 0.0), ('beast', 0.0), ('besid', 0.0), ('berri', 0.0), ('beneath', 0.0), ('bend', 0.0), ('belong', 0.0), ('bell', 0.0), ('believ', 0.0)]
Topic 2:
[('cow', 3.427436461889216), ('creatur', 2.5369695766521796e-09), ('board', 5.0595930924965667e-11), ('bright', 3.975885020692307e-11), ('danger', 3.5361889918675497e-13), ('deal', 1.3369537711553856e-13), ('aliv', 1.1726196188636384e-14), ('cellar', 2.4116117172824423e-15), ('cove', 1.556486667553093e-15), ('curiou', 4.3775063338948756e-16)]
Topic 3:
[('burst', 3.370134935794663), ('board', 1.67095489468018e-11), ('cultiv', 6.260030873759212e-14), ('curiou', 6.485346577221619e-15), ('art', 2.9463980384235572e-15), ('bank', 5.0291651221996e-16), ('daili', 9.349944481859502e-17), ('birch', 7.415

In [25]:
print("LSI Model:")
print_topics(lsi_model, tfidf_vectorizer)
print("=" * 20)

LSI Model:
Topic 0:
[('commenc', 0.9999999999992503), ('blue', 7.639232457802469e-07), ('clear', 5.792938882762372e-07), ('caus', 3.4419012160079515e-07), ('custom', 2.7342775705116873e-07), ('clean', 2.4346426879318273e-07), ('betray', 1.8371094883747335e-07), ('boat', 1.83515437870189e-07), ('care', 1.4457178105967557e-07), ('berri', 1.2656829191697055e-07)]
Topic 1:
[('accord', 0.9999999999972833), ('boat', 1.1277693223294767e-06), ('betray', 9.321143042735877e-07), ('danger', 4.3474945120659707e-07), ('attend', 4.059616045551533e-07), ('cultiv', 3.9233964047406565e-07), ('curiou', 3.3378374790121177e-07), ('death', 2.844353499242574e-07), ('art', 2.6799717683316124e-07), ('daili', 2.454350546957412e-07)]
Topic 2:
[('cow', 0.9999861750948789), ('clear', 0.002292425513548705), ('boat', 0.001423991912573567), ('cultiv', 0.0011439583362818572), ('bright', 0.001119127248411035), ('board', 0.001093868042865346), ('danger', 0.0009729259624226912), ('art', 0.0009723898879935113), ('betray'

In [28]:
print("LDA Model:")
print_topics(lda_model, tfidf_vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('commenc', 552.3207036587878), ('accord', 392.87403334028346), ('care', 75.14476849108297), ('concord', 36.09700588024805), ('buri', 20.19692115780026), ('busi', 17.64346548398226), ('close', 13.235523484191035), ('circumst', 10.11434304835165), ('atmospher', 7.978538925864113), ('chang', 0.1000001935675249)]
Topic 1:
[('cake', 107.20928146020528), ('bring', 106.46982238868219), ('caus', 68.01243973897294), ('buy', 41.60519737224702), ('cultiv', 40.56072114080348), ('bank', 36.91336198224037), ('awak', 30.110261839278827), ('circul', 28.27264823204057), ('behold', 22.79021499248575), ('better', 22.05961407635613)]
Topic 2:
[('art', 44.34590117700969), ('curiou', 43.4203234541159), ('daili', 32.86179963369215), ('degre', 31.379430038394034), ('best', 30.787374297957964), ('cost', 29.000238416759238), ('alreadi', 25.028142672590384), ('admir', 24.4153768493046), ('dark', 20.21390088676006), ('alway', 19.39467999521582)]
Topic 3:
[('burn', 90.8294982092057), ('bright

<a name="5"></a>
### LDA Performance Metrics

In [29]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import imp

print_performance(lda_model, data_vectorized)

Log Likelihood:  -35021.54867560841
Perplexity:  192.9520233059768
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': None,
 'perp_tol': 0.1,
 'random_state': None,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Visualization

In [30]:
data_vectorized = tfidf_vectorizer.fit_transform(post_alice)

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, tfidf_vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  13.302029
6       0.648955  71.707100       2        1  10.026296
1      71.019684   9.971006       3        1  10.003757
4       3.785981  33.185135       4        1   9.643125
3     -42.666500  -9.222070       5        1   9.613073
8      -3.621689 -43.526516       6        1   9.567994
2      -3.632402  -4.171708       7        1   9.515402
5     -36.584400  35.804413       8        1   9.470323
7      44.391380  50.615383       9        1   9.447784
9      40.235809 -31.591665      10        1   9.410218, topic_info=        Term        Freq       Total Category  logprob  loglift
181      say  426.000000  426.000000  Default  30.0000  30.0000
3       alic  303.000000  303.000000  Default  29.0000  29.0000
218    think  134.000000  134.000000  Default  28.0000  28.0000
124    littl  114.000000  114.000000  Default  27.0000  27.0000
128     look  102.000000  102.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
202     soup    0.104512   23.125028  Topic10  -8.6982  -3.0360
52      door    0.104510   29.014393  Topic10  -8.6982  -3.2629
167   rabbit    0.104510   53.859503  Topic10  -8.6982  -3.8815
126  lobster    0.104510   14.499805  Topic10  -8.6982  -2.5692
121     life    0.104510   16.467455  Topic10  -8.6982  -2.6965

[388 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         6  0.958184     add
1         9  0.926499  afraid
2        10  0.975566     air
3         1  0.997872    alic
4         5  0.903802   alway
...     ...       ...     ...
236       4  0.950669    wish
237       5  0.941207     wit
238       5  0.983362  wonder
239       7  0.972973    word
240       9  0.953878   write

[241 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 5, 4, 9, 3, 6, 8, 10])

### Tune Hyperparameters

In [32]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import imp
    
search_params = {'n_components': [5, 15, 30], 'learning_decay': [.5, .9]}
model = GridSearchCV( LatentDirichletAllocation(), param_grid=search_params)
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.9],
                         'n_components': [5, 15, 30]})

### Best Model

In [33]:
best_lda_model = model.best_estimator_
print("Best Model's Params: ", model.best_params_)
print("Best Log Likelihood Score: ", model.best_score_)
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 5}
Best Log Likelihood Score:  -8128.217389611175
Model Perplexity:  184.6006708672652


In [34]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, tfidf_vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  22.877488
3       51.163364 -159.968018       2        1  21.367385
2      -73.144554   21.523815       3        1  19.654397
1      -66.410133 -110.606438       4        1  18.805418
0       54.417324   12.443444       5        1  17.295312, topic_info=       Term        Freq       Total Category  logprob  loglift
181     say  485.000000  485.000000  Default  30.0000  30.0000
3      alic  345.000000  345.000000  Default  29.0000  29.0000
218   think  136.000000  136.000000  Default  28.0000  28.0000
34     come  110.000000  110.000000  Default  27.0000  27.0000
128    look  109.000000  109.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
227  twinkl   14.906826   15.663233   Topic5  -4.3466   1.7052
70      far   14.906826   15.663233   Topic5  -4.3466   1.7052
35   confus   13.777513   14.533920   Topic5  -4.4254   1.7013
175    rest   13.777513   14.533920   Topic5  -4.4254   1.7013
17     best   13.777513   14.533920   Topic5  -4.4254   1.7013

[180 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         4  0.976647      add
3         1  0.997948     alic
5         3  0.974904    anoth
6         3  0.951245   answer
7         3  0.951245  anxious
...     ...       ...      ...
235       1  0.975945    white
236       4  0.976873     wish
238       4  0.976418   wonder
239       3  0.987380     word
240       1  0.941140    write

[150 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 2, 1])

We were able to improve the performance of our topic modeling by tuning the number of components and learning rate, increasing our log-likelihood from -35021 to -8224 and we were able to reduce our perplexity from 193 to 185.  However, to the reader, topics threads remain somewhat perplexing so to speak. As stated in reference below 'optimizing for perplexity may not yield human interpretable topics'.

We downloaded three texts and did an analysis on 'Alice in Wonderland' based on it having less sparsity and content with respect to the others.  It would be interesting to look at how these factors effect the models as well comparison of various pre-processing and vectorization methods.

References:

LDA Performance and Hyperparameter Tuning<br>
https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/


Evaluate Topic Models: Latent Dirichlet Allocation (LDA)<br>
https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0